<a href="https://colab.research.google.com/github/teticio/aventuras-con-textos/blob/master/Amazon_Opiniones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Descargar los datos

In [0]:
import pickle

!wget -q -O train_text.p 'https://docs.google.com/uc?export=download&id=1-0WEtyIuvB8pxENu8WJvgYFb8aMc4F8R'
!wget -q -O train_label.p 'https://docs.google.com/uc?export=download&id=1-AKY4RWMXDg035vTJQLqUXIjYMw6Ja1t'
!wget -q -O test_text.p 'https://docs.google.com/uc?export=download&id=1-4WBoOczIwOpd-Qu5n4dINcor9wC7UUk'

train_text = pickle.load(open('train_text.p', 'rb'))
train_label = pickle.load(open('train_label.p', 'rb'))
test_text = pickle.load(open('test_text.p', 'rb'))

### -----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----

## Tu modelo va aquí

In [0]:
import re

max_len = 512
train_text = [re.sub(r'([^\s\w]|_)+', ' ', _).lower().split() for _ in train_text]
train_text = [_[:max_len] + [''] * (max_len - len(_)) for _ in train_text]

In [0]:
import gensim
from gensim.models.callbacks import CallbackAny2Vec

class logger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 1
        self.loss = 0

    def on_epoch_end(self, model):
        print('#{}'.format(self.epoch), 'loss =',
              (model.get_latest_training_loss() - self.loss) / model.batch_words)
        self.epoch += 1
        self.loss = model.get_latest_training_loss()

embedding_model = gensim.models.Word2Vec(sentences=train_text, size=100, min_count=0, iter=20, compute_loss=True, callbacks=[logger()])
print(embedding_model)

#1 loss = 17.46623125
#2 loss = 12.3455375
#3 loss = 11.770871875
#4 loss = 12.109846875
#5 loss = 10.6608125
#6 loss = 9.73399375
#7 loss = 9.45798125
#8 loss = 8.58353125
#9 loss = 9.05955625
#10 loss = 9.0533
#11 loss = 8.8479625
#12 loss = 8.4974
#13 loss = 8.692625
#14 loss = 8.5485125
#15 loss = 8.3337375
#16 loss = 8.4828375
#17 loss = 8.1151125
#18 loss = 7.9946375
#19 loss = 8.0114125
#20 loss = 8.3201625
Word2Vec(vocab=9005, size=100, alpha=0.025)


In [0]:
import numpy as np

X_train = np.concatenate([np.array([embedding_model.wv.vocab[word].index for word in _])[np.newaxis, :] for _ in train_text], axis=0)
y_train = np.array(train_label)

In [0]:
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense
from keras.layers.embeddings import Embedding

model = Sequential()
model.add(Embedding(
    input_dim=embedding_model.wv.vectors.shape[0],
    output_dim=embedding_model.wv.vectors.shape[1],
    input_length=len(X_train[0]),
    weights=[embedding_model.wv.vectors],
    trainable=False))
model.add(Conv1D(256, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(256, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(256, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Using TensorFlow backend.
W0717 16:37:52.720329 140222969923456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0717 16:37:52.760274 140222969923456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0717 16:37:52.766723 140222969923456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0717 16:37:52.778608 140222969923456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0717 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 512, 100)          900500    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 508, 256)          128256    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 101, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 97, 256)           327936    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 19, 256)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 15, 256)           327936    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
__________

In [0]:
#model.load_weights('AmazonModel.h5')
callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='min'),
             ModelCheckpoint('AmazonModel.h5', save_best_only=True, monitor='val_loss', mode='min')]
# class_weight {'0' : ... , '1' : ... }
model.fit(X_train, 
          y_train,
          validation_split=0.1,
          epochs=100,
          batch_size=512,
          callbacks=callbacks)

Train on 4491 samples, validate on 499 samples
Epoch 1/100
4491/4491 [==============================] - 7s 2ms/step - loss: 1.0843 - acc: 0.7012 - val_loss: 0.5257 - val_acc: 0.8257
Epoch 2/100
4491/4491 [==============================] - 2s 389us/step - loss: 0.4433 - acc: 0.8493 - val_loss: 0.4507 - val_acc: 0.8257
Epoch 3/100
4491/4491 [==============================] - 2s 381us/step - loss: 0.4076 - acc: 0.8519 - val_loss: 0.4502 - val_acc: 0.8257
Epoch 4/100
4491/4491 [==============================] - 2s 384us/step - loss: 0.3961 - acc: 0.8521 - val_loss: 0.4327 - val_acc: 0.8257
Epoch 5/100
4491/4491 [==============================] - 2s 384us/step - loss: 0.3855 - acc: 0.8521 - val_loss: 0.4179 - val_acc: 0.8257
Epoch 6/100
4491/4491 [==============================] - 2s 386us/step - loss: 0.3706 - acc: 0.8528 - val_loss: 0.3918 - val_acc: 0.8236
Epoch 7/100
4491/4491 [==============================] - 2s 382us/step - loss: 0.3437 - acc: 0.8562 - val_loss: 0.3610 - val_acc: 0.8

In [0]:
test_text = [re.sub(r'([^\s\w]|_)+', ' ', _).lower().split() for _ in test_text]
test_text = [_[:max_len] + [''] * (max_len - len(_)) for _ in test_text]

X_test = np.concatenate([np.array([embedding_model.wv.vocab.get(word,embedding_model.wv.vocab['']).index for word in _])[np.newaxis, :] for _ in test_text], axis=0)

model.load_weights('AmazonModel.h5')
test_pred = model.predict(X_test)

### -----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----8<-----

## Tus predicciones deberían tener este formato

- Un numpy array de 2736 probabilidades de que la etiqueta sea '1' por cada uno de las muestras de test_text
- ¡El que menor log_loss (binary_cross_entropy) consigue ganará un premio y entrará en el salón de la fama!
- Manda el fichero pickleado a teticio@gmail.com para obtener tu puntuación

In [0]:
pickle.dump(test_pred, open('test_pred.p', 'wb'))

try:
    from google.colab import files # estamos en colab?
    files.download('test_pred.p') # descargar al ordenador local
except:
    pass

print(len(test_pred))
test_pred

2736


array([[0.9022709 ],
       [0.95577914],
       [0.2206274 ],
       ...,
       [0.81901205],
       [0.90731287],
       [0.9813106 ]], dtype=float32)